# Preparación del dataset para la posterior creación de modelos

Se convierten las variables categóricas a numéricas y se crea un One Hot Encoding de cada una de ellas.

In [2]:
import pandas as pd
import numpy as np
from utils import list_functions as lf
from sklearn.preprocessing import LabelEncoder

#### Para mostrar todas las columnas
pd.set_option('display.max_columns', None, "display.max_columns", None)

In [3]:
df1 = pd.read_csv(r'datasets/studied_benign.csv',low_memory=False)
df2 = pd.read_csv(r'datasets/studied_patho.csv',low_memory=False)

del df1['Unnamed: 0']
del df2['Unnamed: 0'] # Índices de los dataframes anteriores

df1['patho'] = 0
df2['patho'] = 1
df = pd.concat([df1,df2])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34798 entries, 0 to 12166
Columns: 168 entries, ref to patho
dtypes: float64(126), int64(37), object(5)
memory usage: 44.9+ MB


Realizamos una comprobación previa para ver si los dos datasets tienen las mismas columnas, y eliminamos aquellas columnas que en ambos datasets tengan al menos un 50% de nulos.

In [4]:
print(lf.notCommonBothLists(list(df1.columns), list(df2.columns)))

[]


In [5]:
df.dropna(axis=0,subset=['hg19_chr','hg19_pos(1-based)','ref','alt','aaref','aaalt'],inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34787 entries, 0 to 12166
Columns: 168 entries, ref to patho
dtypes: float64(126), int64(37), object(5)
memory usage: 44.9+ MB


## Codificación de las variables categóricas

Convertimos la variable 'hg19_chr' en numérica sustituyendo el valor del cromosoma 'X' por el valor 23

In [6]:
df['hg19_chr'].replace({"X": "23.0"}, inplace=True)
df['hg19_chr'] = df['hg19_chr'].astype(float).astype(int)
print(df['hg19_chr'].unique())

[ 1 10 11 12 13 14 15 16 17 18 19  2 20 21 22  3  4  5  6  7  8  9 23]


Codificamos los valores de las bases de las variables 'ref' y 'alt'

In [7]:
le = LabelEncoder()
le.fit(df[['ref','alt']].stack().unique())

df['ref_code'] = le.transform(df['ref'])
df['alt_code'] = le.transform(df['alt'])

le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{'A': 0, 'C': 1, 'G': 2, 'T': 3}


Codificamos los valores de los aminoácidos de las variables 'aaref' y 'aaalt'

In [8]:
lea = LabelEncoder()
lea.fit(df[['aaref','aaalt']].stack().unique())

df['aaref_code'] = lea.transform(df['aaref'])
df['aaalt_code'] = lea.transform(df['aaalt'])

lea_name_mapping = dict(zip(lea.classes_, lea.transform(lea.classes_)))
print(lea_name_mapping)

{'A': 0, 'C': 1, 'D': 2, 'E': 3, 'F': 4, 'G': 5, 'H': 6, 'I': 7, 'K': 8, 'L': 9, 'M': 10, 'N': 11, 'P': 12, 'Q': 13, 'R': 14, 'S': 15, 'T': 16, 'V': 17, 'W': 18, 'X': 19, 'Y': 20}


Preparamos columnas de One Hot Encoding para las bases, aminoácidos y cromosomas

In [9]:
from sklearn.preprocessing import OneHotEncoder

# OHE de los cromosomas
dummy = pd.get_dummies(df['hg19_chr'], prefix='hg19_chr')
df = pd.concat([df,dummy],axis=1)

# OHE de las bases
dummy = pd.get_dummies(df['ref'], prefix='ref')
df = pd.concat([df,dummy],axis=1)
dummy = pd.get_dummies(df['alt'], prefix='alt')
df = pd.concat([df,dummy],axis=1)

# OHE de los aminoácidos
dummy = pd.get_dummies(df['aaref'], prefix='aaref')
df = pd.concat([df,dummy],axis=1)
dummy = pd.get_dummies(df['aaalt'], prefix='aaalt')
df = pd.concat([df,dummy],axis=1)

## Exportamos el dataset actual (preprocesado)

De esta forma, para posteriores modelos no tener que ejecutar el código anterior

In [10]:
df.to_csv(r"C:\Users\alejs\OneDrive\Documentos\GitHub\TFG\datasets\model_dataset.csv")